# OC Lab — end-to-end demo


## Run the cell below to install dependencies

In [ ]:
!pip install -e ../

### End-to-End OC Harness

This cell runs the **entire OC pipeline**—from Fisher-geometry through Standard-Model RG evolution and the toy FRG flow

---

#### What the harness does
1. **Geometry → Couplings**  
   Monte-Carlo evaluates the Fisher-metric “stiffness” integrals \(I_a\) for U(1), SU(2), SU(3) using the centers and metric settings in the config.  
   From these and the EW observables (αₑₘ, sin²θ₍W₎) it computes the OC scale `x` and the gauge couplings at the UV anchor μ₀.

2. **RG running → MZ**  
   Evolves α₁, α₂, α₃ from μ₀ to 91.1876 GeV with the two-loop SM β-functions (Q3 thresholds).  
   The report compares the resulting α₃(MZ) to the PDG empirical value **0.1181** and flags whether it agrees within ≈ 1 %.

3. **FRG flow → k\***  
   Integrates a toy background-field flow  
   \( \dot\alpha = c\,\alpha^2 \)  
   and stops when |η(α)| = η_freeze (≈ 0.99).  
   It prints both the **numeric** freeze scale k\* and the **analytic expectation** from the ODE solution, checking their log-difference < 0.15.  
   The expected “mid-IR” range is **0.3 – 1.2 GeV**; being in this band is plausible for an onset scale.

4. **OC mass scale & ratio**  
   Computes \(m_\tau\) (internal OC normalization) and the dimensionless ratio  
   \( \mathcal{C} = k_\*/m_\tau \).  
   𝓒 currently serves as a comparative diagnostic, not an external prediction.

5. **Reporting & saving**  
   The cell prints:
   - geometry and MC statistics  
   - EW-lock couplings  
   - α₃(MZ) vs PDG (% difference + status)  
   - FRG parameters and k\* checks  
   - mτ and 𝓒  
   It then saves a timestamped JSON bundle with all numeric and analytic values for provenance.

---

#### How to interpret the report
- ✅ **α₃(MZ ≈ 0.118 ± 1 %)** → SM normalization consistent with experiment.  
- ✅ **η-based stop + small ln-error (< 0.15)** → FRG integration behaving as expected.  
- ✅ **k\* in 0.3–1.2 GeV** → plausible mid-IR freeze scale.  
- 𝓒 ≈ 0.04–0.06 → typical internal ratio for current OC normalization.

⚠️ If any status shows *CHECK*, revisit the corresponding configuration:
- α₃ mismatch → adjust `Ka_group_scale['SU3']`.
- wrong freeze reason → set `alpha_target` ≫ α_cap.
- k\* far outside mid-IR → tweak `growth_c` or η model.

This single cell gives a full empirical vs. model sanity check for a given OC configuration.


In [ ]:
# --- End-to-end OC harness report ---

import yaml, math, time
import numpy as np

from oclab.config import GeometryConfig, EWAnchor, OCParams, RGRun, FRGScan, realize_geometry_centers
from oclab.pipeline import geometry_to_couplings, oc_gap_from_frg
from oclab.frg.flows import expected_kstar_toy
from oclab.io.datasets import bundle_results

# ----------------------- Settings -----------------------
CFG_PATH = "../configs/default.yaml"
PDG_ALPHA3_MZ = 0.1181                 # empirical target for αs(MZ)
MID_IR_BAND   = (0.3, 1.2)             # “plausible” mid-IR k* window (GeV) for toy η

# -------------------- Helper functions ------------------
def pct_diff(meas, ref):
    return 100.0 * (float(meas) - float(ref)) / float(ref)

def status(ok, ok_msg="OK", bad_msg="CHECK"):
    return ("✅ " + ok_msg) if ok else ("⚠️ " + bad_msg)

def fmt(v, prec=6):
    return f"{float(v):.{prec}g}"

# ---------------------- Load config ---------------------
cfg = yaml.safe_load(open(CFG_PATH))
g   = GeometryConfig(**cfg["geometry"])
g = realize_geometry_centers(g)
ew  = EWAnchor(**cfg["ew_anchor"])
oc  = OCParams(**cfg["oc_params"])
rg  = RGRun(**cfg["rg"])
frg = FRGScan(**cfg["frg_scan"])

# ----------------- Geometry → Couplings -----------------
t0 = time.time()
calib, hist = geometry_to_couplings(g, ew, oc, rg)
t1 = time.time()

# RG landing at MZ (final entry):
alpha3_MZ = float(hist.alpha3[-1])
alpha3_mu0 = float(calib.alphas_mu0["SU3"])
delta_pct_alpha3 = pct_diff(alpha3_MZ, PDG_ALPHA3_MZ)
ok_alpha3 = abs(delta_pct_alpha3) <= 1.0  # within ~1% of PDG

# ------------------------ FRG ---------------------------
frg_res, m_tau, C = oc_gap_from_frg(frg, alpha3_mu0, ew.mu0, oc, calib.x)
k_star_num = frg_res.k_star
k_star_exp = expected_kstar_toy(alpha3_mu0, frg.growth_c, frg.kmax, frg.eta_freeze, model=frg.model)
ln_err = abs(math.log(k_star_exp / k_star_num)) if (k_star_num and k_star_exp) else float("inf")

ok_eta_reason = (getattr(frg_res, "reason", None) == "eta_freeze")
ok_ln_match   = (ln_err < 0.15)  # toy ODE vs numeric should agree within ~15% in log-space
ok_k_band     = (MID_IR_BAND[0] <= k_star_num <= MID_IR_BAND[1])

t2 = time.time()

# ----------------------- Report -------------------------
print("OC Harness Report")
print("──────────────────")
print(f"Config     : {CFG_PATH}")
print(f"Timestamp  : {time.strftime('%Y-%m-%d %H:%M:%S')}")
print()

print("Inputs / Anchors")
print("  • μ0 (UV anchor)           :", fmt(ew.mu0), "GeV   (scheme:", ew.scheme, ")")
print("  • α_em(μ0), sin²θW(μ0)     :", fmt(ew.alpha_em_mu0), ",", fmt(ew.sin2_thetaW_mu0))
print()

print("Geometry (Fisher MC) → stiffness I_a and scale x")
print("  • MC samples, seed         :", g.n_samples, ",", g.rng_seed)
print("  • metric model, jitter     :", g.metric_model, ",", g.jitter)
print("  • I_U1, I_SU2, I_SU3       :", fmt(calib.Ia['U1']), ",", fmt(calib.Ia['SU2']), ",", fmt(calib.Ia['SU3']))
print("  • I errors (SE)            :", fmt(calib.Ia_err['U1']), ",", fmt(calib.Ia_err['SU2']), ",", fmt(calib.Ia_err['SU3']))
print("  • x (OC scale)             :", fmt(calib.x))
print()

print("EW lock → couplings at μ0")
print("  • α1(μ0), α2(μ0), α3(μ0)   :",
      fmt(calib.alphas_mu0["U1"]), ",", fmt(calib.alphas_mu0["SU2"]), ",", fmt(alpha3_mu0))
print()

print("RG running → MZ")
print("  • μ_final                  :", fmt(hist.mu[-1]), "GeV (should be 91.1876 GeV)")
print("  • α3(MZ)                   :", fmt(alpha3_MZ),
      f"(target {PDG_ALPHA3_MZ} → {delta_pct_alpha3:+.2f}% )",
      status(ok_alpha3, "α3@MZ ≈ PDG", "α3@MZ off PDG"))
print()

print("FRG (background-field toy, η-based freeze)")
print("  • growth_c, η_freeze, α_cap:", fmt(frg.growth_c), ",", fmt(frg.eta_freeze), ",", fmt(frg.alpha_cap))
print("  • k* numeric (reason)      :", fmt(k_star_num), "GeV", f"({getattr(frg_res,'reason',None)})",
      status(ok_eta_reason, "η-based stop", "stopped by α-target"))
print("  • k* analytic (toy ODE)    :", fmt(k_star_exp), "GeV  (ln|k*_num/k*_exp|=", f"{ln_err:.3f}", ")",
      status(ok_ln_match, "ODE~numeric match", "mismatch"))
print("  • mid-IR plausibility      :", status(ok_k_band, f"in {MID_IR_BAND[0]}–{MID_IR_BAND[1]} GeV",
                                                f"outside {MID_IR_BAND[0]}–{MID_IR_BAND[1]} GeV"))
print()

print("OC mass scale & ratio")
print("  • m_tau                    :", fmt(m_tau), "GeV  (internal OC normalization)")
print("  • 𝓒 = k*/m_tau             :", fmt(C), " (internal diagnostic)")
print()

print("Timing")
print(f"  • geometry+RG: {(t1 - t0):.2f}s   • FRG: {(t2 - t1):.2f}s   • total: {(t2 - t0):.2f}s")
print()

# Write a results bundle with analytic k* recorded
bundle_results({
    "x": calib.x,
    "Ia": calib.Ia,
    "Ia_err": calib.Ia_err,
    "alphas_mu0": calib.alphas_mu0,
    "final_mu": float(hist.mu[-1]),
    "alpha3_final": float(hist.alpha3[-1]),
    "k_star": k_star_num,
    "m_tau": m_tau,
    "C": C,
    "frg": {
        "k_star": k_star_num,
        "reason": getattr(frg_res, "reason", None),
        "reached_alpha": getattr(frg_res, "reached_alpha", None),
        "reached_eta": getattr(frg_res, "reached_eta", None),
        "expected_k_star": k_star_exp,
        "ln_error_expected_vs_numeric": ln_err,
    },
}, prefix="notebook_run_report")


In [ ]:
# --- Procedural, symmetry-locked centers (icosahedral shell) + end-to-end run ---

import numpy as np, yaml, math, time
from copy import deepcopy

from oclab.config import GeometryConfig, EWAnchor, OCParams, RGRun, FRGScan
from oclab.pipeline import geometry_to_couplings, oc_gap_from_frg
from oclab.frg.flows import expected_kstar_toy
from oclab.io.datasets import bundle_results

# ----------------------- Load current config -----------------------
CFG_PATH = "../configs/default.yaml"
cfg = yaml.safe_load(open(CFG_PATH))
g0   = GeometryConfig(**cfg["geometry"])
ew   = EWAnchor(**cfg["ew_anchor"])
oc   = OCParams(**cfg["oc_params"])
rg   = RGRun(**cfg["rg"])
frg  = FRGScan(**cfg["frg_scan"])

# ----------------------- Icosahedral generator ---------------------
def gen_icosa_centers_from_domain(domain, *, A=1.0):
    """
    Return 12 centers on an icosahedral shell with a single, symmetry-locked radius R
    and a single shared width s, both derived from the domain (no extra knobs):

        R := 0.382 * mean half-extent   (0.382 ≈ φ^-2 gives a well-separated shell)
        s := 0.12  * mean full-extent   (≈ old configs' 0.30 when radius≈2.5)

    Each center is [x, y, z, A, s] (A shared; s shared).
    """
    # mean full-extent and half-extent from the box domain [[xL,xH],[yL,yH],[zL,zH]]
    extents   = np.array([hi - lo for lo, hi in domain], float)
    half_ext  = 0.5 * extents
    mean_half = float(np.mean(half_ext))
    mean_full = float(np.mean(extents))

    # symmetry-locked radius/width derived from the domain only (no new hyperparameters)
    R = 0.382 * mean_half       # ≈ 1/φ^2 * mean half-extent
    s = 0.12  * mean_full       # gives s≈0.30 when domain radius≈2.5

    # unit icosa vertices on S^2
    φ = (1 + 5**0.5) / 2
    verts = np.array([
        (0,  1,  φ), (0, -1,  φ), (0,  1, -φ), (0, -1, -φ),
        ( 1,  φ, 0), (-1,  φ, 0), ( 1, -φ, 0), (-1, -φ, 0),
        ( φ, 0,  1), ( φ, 0, -1), (-φ, 0,  1), (-φ, 0, -1),
    ], float)
    verts = (verts / np.linalg.norm(verts, axis=1, keepdims=True)) * R

    centers = [[float(x), float(y), float(z), float(A), float(s)] for x,y,z in verts]
    return centers, R, s

centers_ico, R_used, s_used = gen_icosa_centers_from_domain(g0.domain)

# Replace centers in-memory (keep everything else identical)
g = deepcopy(g0)
g.centers = centers_ico

# ----------------------- Run geometry → RG -------------------------
t0 = time.time()
calib, hist = geometry_to_couplings(g, ew, oc, rg)
t1 = time.time()

# Optional: one-shot normalization to PDG αs(MZ)
PDG_ALPHA3_MZ = 0.1181
def fit_K3_to_alpha3_MZ(oc_in: OCParams):
    # secant-style tiny tuner on Ka_group_scale['SU3']
    def run(scale):
        oc_in.Ka_group_scale = dict(oc_in.Ka_group_scale or {}, SU3=float(scale))
        c2, h2 = geometry_to_couplings(g, ew, oc_in, rg)
        return float(h2.alpha3[-1])
    s0 = (oc.Ka_group_scale or {}).get("SU3", 1.0)
    # proportional first guess
    a_now = float(hist.alpha3[-1])
    s1 = s0 / (PDG_ALPHA3_MZ / a_now)

    a0, a1 = run(s0), run(s1)
    a, b = (s0, a0), (s1, a1)
    for _ in range(8):
        (x0,y0),(x1,y1) = a,b
        x2 = x1 - (y1-PDG_ALPHA3_MZ)*(x1-x0)/((y1-y0) if abs(y1-y0)>1e-12 else 1e-12)
        x2 = float(np.clip(x2, 0.2*s0, 5.0*s0))
        y2 = run(x2)
        a, b = b, (x2, y2)
        if abs(y2 - PDG_ALPHA3_MZ) < 2e-5:
            break
    return b[0]

# Fit SU(3) normalization exactly once to PDG at MZ (keeps the “one datum, one scale” rule)
scale_tuned = fit_K3_to_alpha3_MZ(oc)
calib, hist = geometry_to_couplings(g, ew, oc, rg)
t2 = time.time()

# ----------------------- FRG (η-freeze) ----------------------------
alpha0 = float(calib.alphas_mu0["SU3"])
frg.alpha_target = max(frg.alpha_target, 1e9)  # ensure η-based stop dominates
frg_res, m_tau, C = oc_gap_from_frg(frg, alpha0, ew.mu0, oc, calib.x)
k_star_num = frg_res.k_star
k_star_exp = expected_kstar_toy(alpha0, frg.growth_c, frg.kmax, frg.eta_freeze, model=frg.model)
ln_err = abs(math.log(k_star_exp / k_star_num)) if (k_star_num and k_star_exp) else float("inf")
t3 = time.time()

# ----------------------- Print summary -----------------------------
def fmt(v, p=6): return f"{float(v):.{p}g}" if v is not None else f"{0.0}"
def pct(meas, ref): return 100.0*(float(meas)-float(ref))/float(ref)

print("Centers: icosahedral shell (no per-center tuning)")
print(f"  R (from domain) = {fmt(R_used)}   s (shared) = {fmt(s_used)}   A (shared) = 1.0")
print(f"  N_centers = {len(centers_ico)}")
print()

print("Geometry → EW lock → RG")
print(f"  I_a (U1, SU2, SU3) = {fmt(calib.Ia['U1'])}, {fmt(calib.Ia['SU2'])}, {fmt(calib.Ia['SU3'])}")
print(f"  x = {fmt(calib.x)}")
print(f"  α3(μ0) = {fmt(alpha0)}   α3(MZ) = {fmt(hist.alpha3[-1])}   Δ vs PDG = {pct(hist.alpha3[-1], PDG_ALPHA3_MZ):+.2f}%")
print(f"  Ka_group_scale['SU3'] tuned → {oc.Ka_group_scale.get('SU3')}")
print()

print("FRG (η-freeze)")
print(f"  growth_c={fmt(frg.growth_c)}, η_freeze={fmt(frg.eta_freeze)}, α_cap={fmt(frg.alpha_cap)}")
print(f"  k* (numeric) = {fmt(k_star_num)} GeV  |η|@k* = {fmt(frg_res.reached_eta,4)}  (reason={frg_res.reason})")
print(f"  k* (analytic) = {fmt(k_star_exp)} GeV  ln-error = {ln_err:.3f}")
print(f"  m_tau = {fmt(m_tau)} GeV   𝓒 = {fmt(C)}")
print()

print(f"Timing: geometry {t1-t0:.2f}s, fit {t2-t1:.2f}s, FRG {t3-t2:.2f}s, total {t3-t0:.2f}s")

# ----------------------- Save bundle (optional) --------------------
bundle_results({
    "centers_descriptor": {"type": "icosa", "R": R_used, "A": 1.0, "s": s_used, "N": len(centers_ico)},
    "x": calib.x, "Ia": calib.Ia, "Ia_err": calib.Ia_err,
    "alphas_mu0": calib.alphas_mu0,
    "final_mu": float(hist.mu[-1]), "alpha3_final": float(hist.alpha3[-1]),
    "frg": {"growth_c": frg.growth_c, "eta_freeze": frg.eta_freeze, "alpha_cap": frg.alpha_cap,
            "k_star_numeric": k_star_num, "k_star_expected": k_star_exp,
            "ln_error": ln_err, "reason": getattr(frg_res,"reason",None),
            "reached_eta": getattr(frg_res,"reached_eta",None),
            "reached_alpha": getattr(frg_res,"reached_alpha",None)},
    "m_tau": m_tau, "C": C,
    "Ka_group_scale_SU3": oc.Ka_group_scale.get("SU3")
}, prefix="centers_icosa_run")


In [ ]:
centers_ico

In [ ]:
# --- High-symmetry (and random) centers: uniqueness test ----------------------
import numpy as np, yaml, math, time
from copy import deepcopy

from oclab.config import GeometryConfig, EWAnchor, OCParams, RGRun, FRGScan
from oclab.pipeline import geometry_to_couplings, oc_gap_from_frg
from oclab.frg.flows import expected_kstar_toy
from oclab.io.datasets import write_csv

CFG_PATH = "../configs/default.yaml"
PDG_ALPHA3_MZ = 0.1181

# ---------- Helpers: symmetry-locked center generators (no per-center tuning) ----------
def _radius_width_from_domain(domain):
    extents   = np.array([hi - lo for lo, hi in domain], float)
    half_ext  = 0.5 * extents
    mean_half = float(np.mean(half_ext))
    mean_full = float(np.mean(extents))
    R = 0.382 * mean_half     # φ^-2 * mean half-extent
    s = 0.12  * mean_full     # shared width ~ 0.3 when domain ≈ [-2.5,2.5]^3
    return R, s

def _normalize_shell(verts, R):
    verts = np.asarray(verts, float)
    verts = (verts / np.linalg.norm(verts, axis=1, keepdims=True)) * R
    return verts

def gen_ico(domain, A=1.0):
    φ = (1 + 5**0.5) / 2
    base = [
        (0,  1,  φ), (0, -1,  φ), (0,  1, -φ), (0, -1, -φ),
        ( 1,  φ, 0), (-1,  φ, 0), ( 1, -φ, 0), (-1, -φ, 0),
        ( φ, 0,  1), ( φ, 0, -1), (-φ, 0,  1), (-φ, 0, -1),
    ]
    R, s = _radius_width_from_domain(domain)
    verts = _normalize_shell(base, R)
    C = [[float(x), float(y), float(z), float(A), float(s)] for x,y,z in verts]
    return C, R, s

def gen_dodeca(domain, A=1.0):
    """20 vertices of a regular dodecahedron on a sphere of radius R (shared width s)."""
    φ = (1 + 5**0.5) / 2.0
    invφ = 1.0 / φ

    base = []
    # 8 vertices: (±1, ±1, ±1)
    for sx in (-1, 1):
        for sy in (-1, 1):
            for sz in (-1, 1):
                base.append((sx, sy, sz))

    # 12 vertices: permutations of (0, ±1/φ, ±φ)
    for sy in (-1, 1):
        for sz in (-1, 1):
            base.append((0.0, sy*invφ, sz*φ))   # (0, ±1/φ, ±φ)
    for sx in (-1, 1):
        for sy in (-1, 1):
            base.append((sx*invφ, sy*φ, 0.0))   # (±1/φ, ±φ, 0)
    for sx in (-1, 1):
        for sz in (-1, 1):
            base.append((sx*φ, 0.0, sz*invφ))   # (±φ, 0, ±1/φ)

    # Deduplicate (just in case) and check count
    base = np.array(sorted({(float(x), float(y), float(z)) for (x,y,z) in base}))
    assert base.shape[0] == 20, f"Expected 20 vertices, got {base.shape[0]}"

    # Scale to the shell and attach shared width s
    R, s = _radius_width_from_domain(domain)
    verts = _normalize_shell(base, R)
    C = [[float(x), float(y), float(z), float(A), float(s)] for x, y, z in verts]
    return C, R, s


def gen_octa(domain, A=1.0):
    base = [(1,0,0),(-1,0,0),(0,1,0),(0,-1,0),(0,0,1),(0,0,-1)]
    R, s = _radius_width_from_domain(domain)
    verts = _normalize_shell(base, R)
    C = [[float(x), float(y), float(z), float(A), float(s)] for x,y,z in verts]
    return C, R, s

def gen_tetra(domain, A=1.0):
    base = [(1,1,1),(1,-1,-1),(-1,1,-1),(-1,-1,1)]
    R, s = _radius_width_from_domain(domain)
    verts = _normalize_shell(base, R)
    C = [[float(x), float(y), float(z), float(A), float(s)] for x,y,z in verts]
    return C, R, s

def gen_random_shell(domain, N=12, A=1.0, seed=123):
    R, s = _radius_width_from_domain(domain)
    rng = np.random.default_rng(seed)
    u = rng.normal(size=(N,3))
    u = (u / np.linalg.norm(u, axis=1, keepdims=True)) * R
    C = [[float(x), float(y), float(z), float(A), float(s)] for x,y,z in u]
    return C, R, s

# ---------- Load baseline config & objects ----------
cfg = yaml.safe_load(open(CFG_PATH))
g_base  = GeometryConfig(**cfg["geometry"])
ew_base = EWAnchor(**cfg["ew_anchor"])
oc_base = OCParams(**cfg["oc_params"])
rg_base = RGRun(**cfg["rg"])
frg_base= FRGScan(**cfg["frg_scan"])

# ---------- Step 1: Calibrate SU(3) scale once on Icosa (fit α3(MZ)=PDG) ----------
def fit_K3_on_ico(g, ew, oc, rg):
    C_ico, _, _ = gen_ico(g.domain)
    g1 = deepcopy(g); g1.centers = C_ico
    # secant fit of Ka_group_scale['SU3'] to hit PDG at MZ
    def run(scale):
        oc.Ka_group_scale = dict(oc.Ka_group_scale or {}, SU3=float(scale))
        calib2, hist2 = geometry_to_couplings(g1, ew, oc, rg)
        return float(hist2.alpha3[-1]), calib2
    s0 = (oc.Ka_group_scale or {}).get("SU3", 1.0)
    a_now, _ = run(s0)
    s1 = s0 / (PDG_ALPHA3_MZ / a_now)
    (x0,y0),(x1,y1) = (s0,a_now),(s1,run(s1)[0])
    for _ in range(8):
        if abs(y1-y0) < 1e-12: break
        s2 = x1 - (y1-PDG_ALPHA3_MZ)*(x1-x0)/(y1-y0)
        s2 = float(np.clip(s2, 0.2*s0, 5.0*s0))
        y2,_ = run(s2)
        x0,y0,x1,y1 = x1,y1,s2,y2
        if abs(y2-PDG_ALPHA3_MZ) < 2e-5: break
    oc.Ka_group_scale = dict(oc.Ka_group_scale or {}, SU3=float(x1))
    calib_ico, hist_ico = geometry_to_couplings(g1, ew, oc, rg)
    return float(oc.Ka_group_scale["SU3"]), g1, calib_ico, hist_ico

oc = deepcopy(oc_base)
K3_scale_ico, g_ico, calib_ico, hist_ico = fit_K3_on_ico(deepcopy(g_base), ew_base, oc, rg_base)
alpha0_ico = float(calib_ico.alphas_mu0["SU3"])

# ---------- Step 2: Freeze FRG once (same FRG params) on Icosa (for reference) ----------
frg = deepcopy(frg_base)
# prefer η-based stopping:
frg.alpha_target = max(frg.alpha_target, 1e9)
frg_ico_res, m_tau_ico, C_ico = oc_gap_from_frg(frg, alpha0_ico, ew_base.mu0, oc, calib_ico.x)

# ---------- Step 3: Test other center sets WITHOUT refitting K3 -----------------------
def run_case(name, centers_builder, **kwargs):
    g = deepcopy(g_base)
    g.centers, R, s = centers_builder(g.domain, **kwargs)
    # use SAME oc (with SU3 scale fixed on icosa)
    oc2 = deepcopy(oc)
    # compute geometry→RG
    calib, hist = geometry_to_couplings(g, ew_base, oc2, rg_base)
    alpha0 = float(calib.alphas_mu0["SU3"])
    alpha3_MZ = float(hist.alpha3[-1])
    delta_pct = 100.0*(alpha3_MZ - PDG_ALPHA3_MZ)/PDG_ALPHA3_MZ
    # FRG (η-based)
    frg2 = deepcopy(frg)
    frg_res, m_tau, C = oc_gap_from_frg(frg2, alpha0, ew_base.mu0, oc2, calib.x)
    # analytic toy consistency
    k_exp = expected_kstar_toy(alpha0, frg2.growth_c, frg2.kmax, frg2.eta_freeze, model=frg2.model)
    ln_err = abs(np.log(k_exp / frg_res.k_star)) if frg_res.k_star else float("inf")
    return {
        "name": name, "N": len(g.centers), "R": R, "s": s,
        "I1": calib.Ia["U1"], "I2": calib.Ia["SU2"], "I3": calib.Ia["SU3"],
        "x": calib.x, "alpha3_mu0": alpha0, "alpha3_MZ": alpha3_MZ, "d_pct_MZ": delta_pct,
        "k_star": frg_res.k_star, "reason": getattr(frg_res, "reason", None),
        "eta@k*": getattr(frg_res, "reached_eta", None),
        "k*_exp": k_exp, "ln_err": ln_err, "m_tau": m_tau, "C": C,
    }

cases = []
cases.append(run_case("Icosa (calib)", lambda d: gen_ico(d)))
cases.append(run_case("Dodeca",       lambda d: gen_dodeca(d)))
cases.append(run_case("Octa",         lambda d: gen_octa(d)))
cases.append(run_case("Tetra",        lambda d: gen_tetra(d)))
cases.append(run_case("Random12",     lambda d: gen_random_shell(d, N=12, seed=42)))
cases.append(run_case("Random12b",    lambda d: gen_random_shell(d, N=12, seed=777)))

# ---------- Print concise table ----------
def f(x, p=5):
    try: return f"{float(x):.{p}g}"
    except: return str(x)

headers = ["name","N","alpha3_MZ","Δ% vs PDG","k* [GeV]","|η|@k*","ln|k_num/k_exp|","C","x","I3"]
rowfmt = "{:<14} {:>2}  {:>9}  {:>8}  {:>9}  {:>6}  {:>12}  {:>8}  {:>8}  {:>7}"
print(rowfmt.format(*headers))
for r in cases:
    print(rowfmt.format(
        r["name"], r["N"], f(r["alpha3_MZ"],6),
        f(r["d_pct_MZ"],4), f(r["k_star"],6),
        f(r["eta@k*"],3), f(r["ln_err"],3),
        f(r["C"],4), f(r["x"],5), f(r["I3"],5)
    ))

# ---------- Save CSV for analysis ----------
rows = []
for r in cases:
    rows.append([r["name"], r["N"], r["alpha3_MZ"], r["d_pct_MZ"], r["k_star"],
                 r["eta@k*"], r["k*_exp"], r["ln_err"], r["C"], r["x"], r["I1"], r["I2"], r["I3"]])
path = write_csv(rows, headers=["name","N","alpha3_MZ","d_pct_MZ","k_star","eta_at_kstar",
                                "k_star_expected","ln_err","C","x","I1","I2","I3"],
                 name="frg_centers_uniqueness.csv")
print("\nSaved:", path)
print("Note: SU(3) normalization was fitted **once on Icosa** and reused unchanged for all rows.")


In [1]:
import yaml, math
from oclab.config import GeometryConfig, EWAnchor, OCParams, RGRun, FRGScan, realize_geometry_centers
from oclab.pipeline import geometry_to_couplings, oc_gap_from_frg
from oclab.frg.flows import expected_kstar_toy
import numpy as np

CFG_PATH = "../configs/default.yaml"
cfg = yaml.safe_load(open(CFG_PATH))

g   = realize_geometry_centers(GeometryConfig(**cfg["geometry"]), raw_geometry=cfg["geometry"])

from oclab.oc.stiffness import scaling_exponent_I3_widths
t = scaling_exponent_I3_widths(g)
print("t =", t)

ew  = EWAnchor(**cfg["ew_anchor"])
oc  = OCParams(**cfg["oc_params"])    # Ka_mode should be 'slope'
rg  = RGRun(**cfg["rg"])
frg = FRGScan(**cfg["frg_scan"])

calib, hist = geometry_to_couplings(g, ew, oc, rg)

# After realizing centers and before FRG:
from oclab.oc.stiffness import estimate_stiffness, scaling_exponent_I3_widths
s = scaling_exponent_I3_widths(g)  # robust multi-point fit
I3 = estimate_stiffness(g).I["SU3"]
beta0 = 11 - 2*5/3
K3 = (beta0/(2*np.pi)) / (abs(s) * I3 * calib.x)
alpha3_mu0 = 1.0 / (K3 * I3 * calib.x)
print(f"s={s:.6g}, I3={I3:.6g}, x={calib.x:.6g}, K3={K3:.6g}, alpha3(mu0)={alpha3_mu0:.6g}")

print("x =", calib.x)
print("Ia =", calib.Ia)
print("alpha3(mu0) =", calib.alphas_mu0["SU3"])
print("alpha3(MZ)  =", float(hist.alpha3[-1]))

alpha0 = float(calib.alphas_mu0["SU3"])
frg.alpha_target = max(frg.alpha_target, 1e9)  # prefer η-based stop
frg_res, m_tau, C = oc_gap_from_frg(frg, alpha0, ew.mu0, oc, calib.x)
k_exp = expected_kstar_toy(alpha0, frg.growth_c, frg.kmax, frg.eta_freeze, model=frg.model)
print("k* numeric =", frg_res.k_star, "   k* expected =", k_exp)
print("C =", C)


t = -0.05156485981050466
s=-0.0515649, I3=0.903196, x=52.7059, K3=0.497086, alpha3(mu0)=0.0422598
x = 52.70593931475227
Ia = {'U1': 1.0, 'SU2': 1.046329772473071, 'SU3': 0.9031956569916769}
alpha3(mu0) = 0.052003627902507954
alpha3(MZ)  = 0.0537740144018015
k* numeric = None    k* expected = 0.0016688351782069668
C = nan


In [ ]:
import time, yaml
from oclab.config import GeometryConfig, EWAnchor, OCParams, RGRun, FRGScan, realize_geometry_centers
from oclab.pipeline import geometry_to_couplings

cfg = yaml.safe_load(open("../configs/default.yaml"))
g   = realize_geometry_centers(GeometryConfig(**cfg["geometry"]), raw_geometry=cfg["geometry"])

for backend in ["numpy", "torch_mps"]:
    g.compute = {"backend": backend, "batch_size": 65536, "dtype": "float32"}
    t0 = time.time()
    calib, hist = geometry_to_couplings(g, EWAnchor(**cfg["ew_anchor"]), OCParams(**cfg["oc_params"]), RGRun(**cfg["rg"]))
    dt = time.time()-t0
    print(f"{backend}: x={calib.x:.6g}, I3={calib.Ia['SU3']:.6g}, time={dt:.2f}s")


In [ ]:
import torch
print("MPS available:", torch.backends.mps.is_available())
print("Built w/ MPS:", torch.backends.mps.is_built())

# Create a tensor via your backend shim's path:
from oclab.compute.backend import get_backend, to_device
bk = get_backend("torch_mps", dtype="float32")
t = to_device(bk.xp, [[1.0,2.0,3.0]], bk.device)
print("tensor device:", t.device if hasattr(t, "device") else "numpy")


In [ ]:
import time, numpy as np, yaml
from oclab.config import GeometryConfig, EWAnchor, OCParams, RGRun, realize_geometry_centers
from oclab.pipeline import geometry_to_couplings

def run_once(backend="numpy", n_samples=160_000, batch_size=65_536):
    cfg = yaml.safe_load(open("../configs/default.yaml"))
    g = realize_geometry_centers(GeometryConfig(**cfg["geometry"]), raw_geometry=cfg["geometry"])
    g.n_samples = int(n_samples)
    g.compute = {"backend": backend, "batch_size": int(batch_size), "dtype": "float32"}
    ew, oc, rg = EWAnchor(**cfg["ew_anchor"]), OCParams(**cfg["oc_params"]), RGRun(**cfg["rg"])

    t0 = time.time()
    calib, hist = geometry_to_couplings(g, ew, oc, rg)
    t1 = time.time()
    print(f"[{backend}] N={n_samples:,} batch={batch_size}  x={calib.x:.6g}  I3={calib.Ia['SU3']:.6g}  time={t1-t0:.2f}s")
    return calib, hist, t1-t0

# Try the three backends with the same N and batch:
for bk in ["numpy", "torch_cpu", "torch_mps"]:
    run_once(bk, n_samples=160_000, batch_size=65_536)


In [ ]:
for N in [160_000, 1_000_000, 5_000_000]:
    for bk in ["numpy", "torch_mps"]:
        run_once(bk, n_samples=N, batch_size=131_072)
    print("-" * 60)
